In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12089449
paper_name = 'jorgensen_tyers_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
sheets = ['haps','hets']
data_cols = [2, 1]

In [15]:
original_data_list = []
for ixs, s in enumerate(sheets):
    original_data = pd.read_excel('raw_data/data.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    
    original_data['orf'] = original_data['ORF'].astype(str)
    
    original_data['orf'] = clean_orf(original_data['orf'])
    
    # N/A seems to be the WT in het based on the description of the data in the paper
    original_data.loc[original_data['orf']=='NAN','orf'] = 'WT'
    
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = pd.to_numeric(original_data.iloc[:,data_cols[ixs]], errors='coerce')
    
    original_data.set_index('orf', inplace=True)
    
    original_data = original_data[['data']].copy()
    
    # Normalize by WT
    original_data = original_data.div(original_data.loc['WT',:], axis=1) - 1
    
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 399 x 3
       ORF  Daughter cell size (fL)  Median cell size (fL)
0  YDL049C                      7.0                   21.0
1  YDR470C                      7.0                   32.0
2  YLR403W                      9.0                   25.0
3  YHR205W                      9.0                   27.0
4  YKL138C                     10.0                   29.0
            ORF  Daughter cell size (fL)  Median cell size (fL) orf
index_input                                                        
186          WT                     23.0                   41.0  WT
(389, 1)
Original data dimensions: 54 x 2
       ORF  Median cell size (fL)
0  YAL041W                     61
1  YOL120C                     62
2  YOL040C                     63
3  YBR091C                     63
4  YOR341W                     64
             ORF  Median cell size (fL) orf
index_input                                
25           NaN                     70  WT
(54, 1)


In [16]:
original_data = pd.concat(original_data_list, axis=1)

In [18]:
original_data.drop(index='WT',inplace=True)

In [20]:
original_data.index.name='orf'

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [68, 709]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,68,709
data_type,value,value
orf,,
YAL010C,-0.097561,NaN
YAL016W,-0.195122,NaN
YAL021C,0.463415,NaN
YAL035W,0.097561,NaN
YAL040C,0.439024,NaN


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,68,709
,data_type,value,value
gene_id,orf,,
8,YAL010C,-0.097561,NaN
14,YAL016W,-0.195122,NaN
19,YAL021C,0.463415,NaN
33,YAL035W,0.097561,NaN
38,YAL040C,0.439024,NaN


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,68,709,68,709
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
8,YAL010C,-0.097561,NaN,-0.836136,NaN
14,YAL016W,-0.195122,NaN,-1.740808,NaN
19,YAL021C,0.463415,NaN,4.365728,NaN
33,YAL035W,0.097561,NaN,0.973208,NaN
38,YAL040C,0.439024,NaN,4.139560,NaN


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 12089449...
Inserting the new data...


100%|██████████| 2/2 [00:01<00:00,  1.34it/s]

Updating the data_modified_on field...
